In [193]:
# Libraries for Web Scraping
from bs4 import BeautifulSoup
import requests

# Libraries for Data Processing
import pandas as pd 
from unidecode import unidecode
from datetime import datetime
import time
import re

# Récupération des données des sets pokémons (Nom du set + cartes associées)

In [194]:
pokemon_set_name_list =[
    'PAF',
    'PAR',
    'MEW',
    'OBF',
    'PAL',
    'SVI',
    'CRZ',
    'SIT',
    'LOR',
    'PGO',
    'ASR',
    'BRS',
    'FST',
    'CEL',
    'EVS',
    'CRS',
    'SWSH5',
    'SWSH45',
    'SWSH4',
    'SWSH35',
    'SWSH3',
    'SWSH2',
    'SWSH1',
    ]

In [195]:
def get_set_name_data(pokemon_set_name_list):
    soup_list = []

    for set_name in pokemon_set_name_list:
        url = f'https://www.pokecardex.com/series/{set_name}'
        
        # Récupérer le contenu de la page
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            soup_list.append(soup)
        else:
            print(f"La requête pour {set_name} a échoué avec le code d'état {r.status_code}")

    return soup_list

In [196]:
def parse(soup_list):
    set_data_list = []


    for soup in soup_list:
        
        # Trouver le titre de la série s'il est présent
        set_title_tag = soup.find('h5', class_='d-block text-center mb-3')
        if set_title_tag:
            set_title = set_title_tag.text.strip()

            # Trouver toutes les divisions de carte sur la page
            card_divs = soup.find_all('div', class_='serie-details-carte')

            # Pour chaque division de carte, extraire le titre de la carte et l'associer à la série
            for card_div in card_divs:
                card_title = card_div.find('a').get('title')
                set_data_list.append({'set_name': set_title, 'card_title': card_title})
        else:
            print("Balise de titre de série non trouvée sur la page.")

    return  set_data_list

In [197]:
def output(set_data_list):
    cards_df = pd.DataFrame(set_data_list)
    cards_df.drop_duplicates(inplace=True)
    return cards_df

In [198]:
def set_name_scrap(pokemon_set_name_list):
    soup = get_set_name_data(pokemon_set_name_list)
    soup_list = parse(soup)
    cards_df = output(soup_list)
    cards_df['card_title'] = cards_df['card_title'].apply(unidecode)
    cards_df['card_title'] = cards_df['card_title'].str.lower()
    cards_df['card_title'] = cards_df['card_title'].str.replace('/', ' ')
    cards_df['card_title'] = cards_df['card_title'].str.replace('-', ' ')
    return cards_df

In [199]:
pokemon_set_df = set_name_scrap(pokemon_set_name_list)

Balise de titre de série non trouvée sur la page.


In [200]:
pokemon_set_df

,set_name,card_title
0,Destinées de Paldea,pomdepik 1 91
1,Destinées de Paldea,foretress ex 2 91
2,Destinées de Paldea,maracachi 3 91
3,Destinées de Paldea,terracool 4 91
4,Destinées de Paldea,terracruel ex 5 91
...,...,...
4610,Épée et Bouclier,zamazenta v 212 202
4611,Épée et Bouclier,ballon 213 202
4612,Épée et Bouclier,ecusson metal 214 202
4613,Épée et Bouclier,canne ordinaire 215 202


# Récupération du nom des Blocs ( Ecarlate et Violet, Epée et Bouclier, etc...)

In [201]:
def get_bloc_name_data():
    soup_list = []

    
    url = f'https://www.pokecardex.com/series/'
        
    # Récupérer le contenu de la page
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        soup_list.append(soup)
    else:
        print(f"La requête pour {set_name} a échoué avec le code d'état {r.status_code}")

    return soup_list

In [202]:
def bloc_name_parse(soup_list):
    bloc_names_list = []
    for soup in soup_list:
        
        bloc_name_tags = soup.find_all('p', class_='p-0 m-0')
        for bloc_tag in bloc_name_tags:
            bloc_name = bloc_tag.text.strip()
            bloc_names_list.append(bloc_name)
    return bloc_names_list

In [203]:
def bloc_name_list():
    bloc_soup_list = get_bloc_name_data()
    bloc_name_list = bloc_name_parse(bloc_soup_list)
    
    # Liste pour stocker les noms de blocs modifiés
    modified_bloc_name_list = []
    
    for bloc_name in bloc_name_list:
        # Convertir en minuscules et enlever les accents
        modified_bloc_name = unidecode(bloc_name.lower())
        modified_bloc_name_list.append(modified_bloc_name)
    
    modified_bloc_name_list = modified_bloc_name_list[:2]
        
    return modified_bloc_name_list

In [204]:
bloc_name_list = bloc_name_list()

In [205]:
bloc_name_list 

['ecarlate et violet', 'epee et bouclier']

# Assignation du nom du bloc aux séries correspondantes

In [206]:
set_name_list = pokemon_set_df['set_name'].unique()

### Création de listes distincetes en fonction des séries

In [207]:
SV_set_name_list = set_name_list[:6]
EB_set_name_list = set_name_list[6:]

In [208]:
SV_set_name_list

array(['Destinées de Paldea', 'Faille Paradoxe', '151',
       'Flammes Obsidiennes', 'Évolutions à Paldea', 'Écarlate et Violet'],
      dtype=object)

In [209]:
EB_set_name_list

array(['Zénith Suprême', 'Tempête Argentée', 'Origine Perdue',
       'Pokémon GO', 'Astres Radieux', 'Stars Étincelantes',
       'Poing de Fusion', 'Célébrations', 'Évolution Céleste',
       'Styles de Combat', 'Destinées Radieuses', 'Voltage Éclatant',
       'La Voie du Maître', 'Ténèbres Embrasées', 'Clash des Rebelles',
       'Épée et Bouclier'], dtype=object)

### Attribution du bloc correspondant aux séries

In [210]:
for sv_set_name in SV_set_name_list:
    
    filtered_rows = pokemon_set_df['set_name'] == sv_set_name
    
    pokemon_set_df.loc[filtered_rows, 'bloc_name'] = bloc_name_list[0]

In [211]:
for eb_set_name in EB_set_name_list:
    
    filtered_rows = pokemon_set_df['set_name'] == eb_set_name
    
    pokemon_set_df.loc[filtered_rows, 'bloc_name'] = bloc_name_list[1]

In [218]:
pokemon_set_df.to_csv(r"C:\Users\Jean_Professionel\OneDrive - ESEO\Documents\csv_data",index=False)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Jean_Professionel\\OneDrive - ESEO\\Documents\\csv_data'